In [ ]:
# imporing lib..................
import os 
from dataclasses import dataclass ,field 
from typing import dict , Optional 
import torch 
from datasets import Dataset , load_dataset 
from peft import AutoPeftModelForCausalLM,LoraConfig 
from transformers import AutoTokenizer ,HfArgumentParser ,TrainingArguments

from trl import DPOTrainer 

In [ ]:
# DATSEt preocessing 
def dpo_data(train_or_val):
    dataset = load_dataset(
        "Dahoas/full-hh-rlhf",
        split="train",
        use_auth_token=True
    )
    original_columns = dataset.column_names 
    

    def return_prompt_and_responses(samples):
        return{
            "prompt":[prompt for prompt in samples["prompt"]],
            "choosen":samples["choosen"]
            "rejected":samples["rejected"]
        }
    return dartaset.map(
        return_prompt_and_responses,
        batched=True,
        remove_columsn=original_columns
    )